### Quick Start

The `ssms` package serves two purposes. 

1. Easy access to *fast simulators of sequential sampling models*
   
2. Support infrastructure to construct training data for various approaches to likelihood / posterior amortization

We provide two minimal examples here to illustrate how to use each of the two capabilities.


#### Install 

Let's start with *installing* the `ssms` package.

You can do so by typing,

`pip install git+https://github.com/AlexanderFengler/ssm_simulators`

in your terminal.

Below you find a basic tutorial on how to use the package.

#### Tutorial

In [1]:
# Import necessary packages
import numpy as np
import pandas as pd
import ssms

#### Using the Simulators

Let's start with using the basic simulators. 
You access the main simulators through the  `ssms.basic_simulators.simulator` function.

To get an idea about the models included in `ssms`, use the `config` module.
The central dictionary with metadata about included models sits in `ssms.config.model_config`. 

In [2]:
# Check included models
list(ssms.config.model_config.keys())[:10]

['ddm',
 'ddm_legacy',
 'angle',
 'weibull',
 'levy',
 'levy_angle',
 'full_ddm',
 'gamma_drift',
 'gamma_drift_angle',
 'ds_conflict_drift']

In [3]:
# Take an example config for a given model
ssms.config.model_config['ddm']

{'name': 'ddm',
 'params': ['v', 'a', 'z', 't'],
 'param_bounds': [[-3.0, 0.3, 0.1, 0.0], [3.0, 2.5, 0.9, 2.0]],
 'boundary': <function ssms.basic_simulators.boundary_functions.constant(t=0)>,
 'n_params': 4,
 'default_params': [0.0, 1.0, 0.5, 0.001],
 'hddm_include': ['z'],
 'nchoices': 2}

**Note:**
The usual structure of these models includes,

- Parameter names (`'params'`)
- Bounds on the parameters (`'param_bounds'`)
- A function that defines a boundary for the respective model (`'boundary'`)
- The number of parameters (`'n_params'`)
- Defaults for the parameters (`'default_params'`)
- The number of choices the process can produce (`'nchoices'`)

The `'hddm_include'` key concerns information useful for integration with the [hddm](https://github.com/hddm-devs/hddm) python package, which facilitates hierarchical bayesian inference for sequential sampling models. It is not important for the present tutorial.

In [4]:
from ssms.basic_simulators import simulator
sim_out = simulator(model = 'ddm', 
                    theta = [0, 1, 0.5, 0.5],
                    n_samples = 1000)

In [5]:
np.hstack([sim_out['rts'], sim_out['choices']])

array([[ 1.36599243,  1.        ],
       [ 0.9799974 , -1.        ],
       [ 1.62999678, -1.        ],
       ...,
       [ 0.9099983 ,  1.        ],
       [ 0.80099964,  1.        ],
       [ 1.21699429,  1.        ]])

The output of the simulator is a `dictionary` with three elements.

1. `rts` (array)
2. `choices` (array)
3. `metadata` (dictionary)

The `metadata` includes the named parameters, simulator settings, and more.

#### Using the Training Data Generators

The training data generators sit on top of the simulator function to turn raw simulations into usable training data for training machine learning algorithms aimed at posterior or likelihood armortization.

We will use the `data_generator` class from `ssms.dataset_generators`. Initializing the `data_generator` boils down to supplying two configuration dictionaries.

1. The `generator_config`, concerns choices as to what kind of training data one wants to generate.
2. The `model_config` concerns choices with respect to the underlying generative *sequential sampling model*. 

We will consider a basic example here, concerning data generation to prepare for training [LANs](https://elifesciences.org/articles/65074).

Let's start by peeking at an example `generator_config`.

In [6]:
ssms.config.data_generator_config['lan']['mlp']

{'output_folder': 'data/lan_mlp/',
 'dgp_list': 'ddm',
 'nbins': 0,
 'n_samples': 100000,
 'n_parameter_sets': 10000,
 'n_parameter_sets_rejected': 100,
 'n_training_samples_by_parameter_set': 1000,
 'max_t': 20.0,
 'delta_t': 0.001,
 'pickleprotocol': 4,
 'n_cpus': 'all',
 'kde_data_mixture_probabilities': [0.8, 0.1, 0.1],
 'simulation_filters': {'mode': 20,
  'choice_cnt': 0,
  'mean_rt': 17,
  'std': 0,
  'mode_cnt_rel': 0.9},
 'negative_rt_cutoff': -66.77497,
 'n_subruns': 10,
 'bin_pointwise': False,
 'separate_response_channels': False}

You usually have to make just few changes to this basic configuration dictionary.
An example below.

In [7]:
from copy import deepcopy
# Initialize the generator config (for MLP LANs)
generator_config = deepcopy(ssms.config.data_generator_config['snpe'])
# Specify generative model (one from the list of included models mentioned above)
generator_config['dgp_list'] = 'angle' 
# Specify number of parameter sets to simulate
generator_config['n_parameter_sets'] = 100 
# Specify how many samples a simulation run should entail
generator_config['n_samples'] = 1000

Now let's define our corresponding `model_config`.

In [8]:
model_config = ssms.config.model_config['angle']
print(model_config)

{'name': 'angle', 'params': ['v', 'a', 'z', 't', 'theta'], 'param_bounds': [[-3.0, 0.3, 0.1, 0.001, -0.1], [3.0, 3.0, 0.9, 2.0, 1.3]], 'boundary': <function angle at 0x11bd60b80>, 'n_params': 5, 'default_params': [0.0, 1.0, 0.5, 0.001, 0.0], 'hddm_include': ['z', 'theta'], 'nchoices': 2}


We are now ready to initialize a `data_generator`, after which we can generate training data using the `generate_data_training_uniform` function, which will use the hypercube defined by our parameter bounds from the `model_config` to uniformly generate parameter sets and corresponding simulated datasets.

In [9]:
my_dataset_generator = ssms.dataset_generators.data_generator_snpe(generator_config = generator_config,
                                                                   model_config = model_config)

n_cpus used:  6
checking:  data/snpe_training/


In [10]:
training_data = my_dataset_generator.generate_data_training_uniform(save = True)

simulation round: 1  of 10
simulation round: 2  of 10
simulation round: 3  of 10
simulation round: 4  of 10
simulation round: 5  of 10
simulation round: 6  of 10
simulation round: 7  of 10
simulation round: 8  of 10
simulation round: 9  of 10
simulation round: 10  of 10
Writing to file:  data/snpe_training/training_data__n_1000/angle/training_data_angle_4c70e020dace11ec9074acde48001122.pickle


In [8]:
new_features = {i: {'data': training_data[0][i]['features'], 'labels': training_data[0][i]['labels']} for i in range(len(training_data[0]))}

In [12]:
training_data.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [ ]:
trainin

In [17]:
training_data[0]

[{'features: ': array([[ 4.10223436,  1.        ],
         [ 2.655339  ,  1.        ],
         [ 3.40328479,  1.        ],
         ...,
         [ 2.71133494,  1.        ],
         [ 0.60232329, -1.        ],
         [ 1.18331599, -1.        ]]),
  'labels': array([ 1.2677877 ,  2.0692544 ,  0.17184597,  0.36032298, -0.06370651],
        dtype=float32),
  'meta': {'v': array([1.2677877], dtype=float32),
   'a': array([2.0692544], dtype=float32),
   'z': array([0.17184597], dtype=float32),
   't': array([0.36032298], dtype=float32),
   's': 1.0,
   'theta': array([-0.06370651], dtype=float32),
   'delta_t': 0.0010000000474974513,
   'max_t': 20.0,
   'n_samples': 1000,
   'simulator': 'ddm_flexbound',
   'boundary_fun_type': 'angle',
   'possible_choices': [-1, 1],
   'trajectory': array([[  -1.3580683],
          [  -1.3061538],
          [  -1.3130792],
          ...,
          [-999.       ],
          [-999.       ],
          [-999.       ]], dtype=float32),
   'boundary': arr

In [20]:
max_n_trials = 3000
mydict = {0: {'features': np.zeros((max_n_trials, 2)), 'labels': np.ones(4)},
          1: {'features': np.zeros((max_n_trials, 2)), 'labels': np.ones(4)}}


n_trials = int(np.random.uniform(low = 500, high = 3000))
n_batch = 2

# Inside the dataloader
my_batch = np.zeros((n_batch, n_trials, 2))

for i in range(n_batch):
    my_batch[i, :, :] = mydict[i]['features'][np.random.choice(max_n_trials, n_trials, replace = False), :]

In [21]:
my_batch.shape

(2, 1488, 2)

In [14]:
np.random.choice(10, 2, replace=False)

array([4, 5])

`training_data` is a dictionary containing four keys:

1. `data` the features for [LANs](https://elifesciences.org/articles/65074), containing vectors of *model parameters*, as well as *rts* and *choices*.
2. `labels` which contain approximate likelihood values
3. `generator_config`, as defined above
4. `model_config`, as defined above

You can now use this training data for your purposes. If you want to train [LANs](https://elifesciences.org/articles/65074) yourself, you might find the [LANfactory](https://github.com/AlexanderFengler/LANfactory) package helpful.

You may also simply find the basic simulators provided with the **ssms** package useful, without any desire to use the outputs into training data for amortization purposes.

##### END